In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
# 从文件加载txt文件
loader = TextLoader('mla.txt', encoding='utf-8')
documents = loader.load()

# 切分txt文档
text_splitter = CharacterTextSplitter(
    separator="###", 
    chunk_size=2000, # 因为是按整首歌来切，所以可以把块的大小设置得大一些
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)
split_documents = text_splitter.split_documents(documents)


# 创建嵌入模型
# 自动从网上下载一个开源的嵌入模型到电脑上
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# 步骤四：创建LLM
llm = ChatOllama(model='llama3.1:8b') 
# 步骤五：创建向量数据库
vector_store = FAISS.from_documents(split_documents, embeddings)
#print("向量数据库创建成功！")

# 创建检索器
retriever = vector_store.as_retriever()

# 创建提示模板
prompt = ChatPromptTemplate.from_template("""
请只根据下面提供的上下文来回答用户的问题。如果上下文中没有相关信息，就回答“根据您提供的资料，我无法回答这个问题”。

<context>
{context}
</context>

问题: {input}
""")

# 创建并组装链条
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

print("My Little Airport歌词问答链条组装完成！可以开始提问了。")


# 开始提问
# 替换成你想问的、和mla.txt 文件内容相关的问题
response = retrieval_chain.invoke({"input": "请问悲傷的採購的歌词是什么？"})
print("AI的回答:", response["answer"])

My Little Airport歌词问答链条组装完成！可以开始提问了。
AI的回答: 根据您提供的资料，我可以回答这个问题。悲傷的採購（"The Melancholic Shopping"）的一部分歌词如下：

“订了一季，又到下季。
為何人大了就要成為工作的奴隸，最愛作的不可發揮。
我的感覺，逐漸流逝。
從前曾話過要如何欣賞世界的美麗，現在只懂得放假去消費。如果繼續這樣，我還是我嗎？我還喜歡我嗎？
或者我應該死去吧，總好過變得更差。如果繼續這樣，你還願意嗎？
和我奏著結他？日子不應該這樣吧，而現實就是這樣可怕。如果繼續這樣……”
